## Build Functions for ELT

In [2]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [3]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import functions as f

%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [4]:
# manufactoring suppliers 
# 'lrcs', 'klac', ''

In [10]:
f.predictions('amd')

AMD 5m Interval Timestamp: 2024-10-29 03:51:23 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,up
kelly_1:2.5,0.475,0.791212,0.778952,0.779747
prob_up,0.625,0.850865,0.842109,0.842676
prob_static,0.375,0.14835,0.150294,0.146948
prob_down,0.0,0.000785,0.007597,0.010375
precision,0.796421,0.802721,0.811189,0.653422
recall,0.746331,0.742138,0.72956,0.620545
f1,0.770563,0.771242,0.768212,0.636559
support,"[477.0, 479.0, 477.0]","[477.0, 479.0, 477.0]","[477.0, 479.0, 477.0]","[477.0, 479.0, 477.0]"


AMD 15m Interval Timestamp: 2024-10-29 03:51:30 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,up
kelly_1:2.5,0.475,0.637989,0.403045,0.529759
prob_up,0.625,0.741421,0.573603,0.664114
prob_static,0.375,0.254962,0.404622,0.289332
prob_down,0.0,0.003618,0.021775,0.046554
precision,0.759259,0.75,0.751553,0.60241
recall,0.773585,0.754717,0.761006,0.628931
f1,0.766355,0.752351,0.75625,0.615385
support,"[159.0, 160.0, 156.0]","[159.0, 160.0, 156.0]","[159.0, 160.0, 156.0]","[159.0, 160.0, 156.0]"


AMD 30m Interval Timestamp: 2024-10-29 03:51:34 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,up,up,up
kelly_1:2.5,0.3,0.96987,0.987441,0.811446
prob_up,0.5,0.978479,0.991029,0.865319
prob_static,0.5,0.021056,0.007491,0.107957
prob_down,0.0,0.000465,0.00148,0.026724
precision,0.700337,0.784722,0.784173,0.720165
recall,0.709898,0.771331,0.744027,0.59727
f1,0.705085,0.777969,0.763573,0.652985
support,"[293.0, 293.0, 291.0]","[293.0, 293.0, 291.0]","[293.0, 293.0, 291.0]","[293.0, 293.0, 291.0]"


AMD 1wk Interval Timestamp: 2024-10-29 03:51:50 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.3,0.555053,0.740723,0.447372
prob_up,0.375,0.274057,0.145586,0.258355
prob_static,0.5,0.682181,0.814802,0.605265
prob_down,0.125,0.043762,0.039612,0.13638
precision,0.823529,0.795918,0.813559,0.661017
recall,0.724138,0.672414,0.827586,0.672414
f1,0.770642,0.728972,0.820513,0.666667
support,"[59.0, 58.0, 56.0]","[59.0, 58.0, 56.0]","[59.0, 58.0, 56.0]","[59.0, 58.0, 56.0]"


AMD 1mo Interval Timestamp: 2024-10-29 03:51:52 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,static,static,static
kelly_1:2.5,0.125,0.976413,0.951075,0.727027
prob_up,0.375,0.012083,0.01504,0.060127
prob_static,0.25,0.983152,0.965054,0.805019
prob_down,0.375,0.004765,0.019907,0.134853
precision,0.705882,0.909091,0.952381,0.653846
recall,0.857143,0.769231,0.769231,0.653846
f1,0.774194,0.833333,0.851064,0.653846
support,"[28.0, 26.0, 26.0]","[28.0, 26.0, 26.0]","[28.0, 26.0, 26.0]","[28.0, 26.0, 26.0]"


### Check Summary Output

In [ ]:
symbol='NVDA'
interval='5m'

# Define Eastern Time Zone
eastern = pytz.timezone('US/Eastern')

# Get current time in Eastern Time Zone
eastern_time = datetime.now(eastern)

# Format the time to include hour, minute, and seconds
time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [27]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')